In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install catboost
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [3]:
df_train = pd.read_csv('train_IxoE5JN.csv')
df_train.tail(30)

,row_id,datetime,energy
94962,94963,2018-12-30 18:00:00,2414.6892
94963,94964,2018-12-30 19:00:00,2364.2212
94964,94965,2018-12-30 20:00:00,2285.9276
94965,94966,2018-12-30 21:00:00,2180.8996
94966,94967,2018-12-30 22:00:00,2134.6600
94967,94968,2018-12-30 23:00:00,1998.5328
94968,94969,2018-12-31 00:00:00,1827.8964
94969,94970,2018-12-31 01:00:00,1713.0476
94970,94971,2018-12-31 02:00:00,1619.6136
94971,94972,2018-12-31 03:00:00,1555.6420


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   row_id    94992 non-null  int64  
 1   datetime  94992 non-null  object 
 2   energy    93092 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [5]:
df_train['energy'] = df_train['energy'].fillna(method='ffill')
df_train.head(3)

,row_id,datetime,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500
2,3,2008-03-01 02:00:00,1056.247500


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   row_id    94992 non-null  int64  
 1   datetime  94992 non-null  object 
 2   energy    94992 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [7]:
df_train.rename(columns = {'datetime':'date_time'}, inplace = True)
df_train.head(3)

,row_id,date_time,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500
2,3,2008-03-01 02:00:00,1056.247500


In [8]:
df_train['year'] = df_train['date_time'].str[:4]
df_train['year'] = df_train['year'].astype('int64')

df_train['month'] = df_train['date_time'].str[5:7]
df_train['month'] = df_train['month'].astype('int64')

df_train['day'] = df_train['date_time'].str[8:10]
df_train['day'] = df_train['day'].astype('int64')

df_train['hour'] = df_train['date_time'].str[11:13]
df_train['hour'] = df_train['hour'].astype('int64')

df_train.head(3)

,row_id,date_time,energy,year,month,day,hour
0,1,2008-03-01 00:00:00,1259.985563,2008,3,1,0
1,2,2008-03-01 01:00:00,1095.541500,2008,3,1,1
2,3,2008-03-01 02:00:00,1056.247500,2008,3,1,2


In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   row_id     94992 non-null  int64  
 1   date_time  94992 non-null  object 
 2   energy     94992 non-null  float64
 3   year       94992 non-null  int64  
 4   month      94992 non-null  int64  
 5   day        94992 non-null  int64  
 6   hour       94992 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 5.1+ MB


In [10]:
df_train['date_time'] = pd.to_datetime(df_train['date_time'])

In [11]:
df_train['weekday'] = df_train['date_time'].dt.dayofweek
df_train.drop(['date_time', 'row_id'], axis = 1, inplace = True)
df_train.head(3)

,energy,year,month,day,hour,weekday
0,1259.985563,2008,3,1,0,5
1,1095.541500,2008,3,1,1,5
2,1056.247500,2008,3,1,2,5


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   energy   94992 non-null  float64
 1   year     94992 non-null  int64  
 2   month    94992 non-null  int64  
 3   day      94992 non-null  int64  
 4   hour     94992 non-null  int64  
 5   weekday  94992 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 4.3 MB


In [13]:
from sklearn.model_selection import train_test_split

X = df_train.drop(columns = 'energy', axis = 1)
y = df_train['energy']

(X_train, X_val, y_train, y_val) = train_test_split(X, y, test_size = 0.1)

In [14]:
X_train.head(3)

,year,month,day,hour,weekday
93825,2018,11,13,9,1
72668,2016,6,14,20,1
79501,2017,3,26,13,6


In [15]:
X_val.head(3)

,year,month,day,hour,weekday
70899,2016,4,2,3,5
31996,2011,10,25,4,1
57280,2014,9,12,16,4


In [16]:
from catboost import CatBoostRegressor as cbr

cat = cbr(iterations=3000, learning_rate=0.01, depth=7, l2_leaf_reg= 3, loss_function='RMSE') 

catreg = cat.fit(X_train,y_train,verbose_eval=200,plot=True,eval_set=(X_val,y_val))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 366.9491716	test: 366.2586329	best: 366.2586329 (0)	total: 171ms	remaining: 8m 34s
200:	learn: 187.9069482	test: 187.7401766	best: 187.7401766 (200)	total: 4.7s	remaining: 1m 5s
400:	learn: 160.2789979	test: 160.3621134	best: 160.3621134 (400)	total: 10.1s	remaining: 1m 5s
600:	learn: 148.1003025	test: 148.4162524	best: 148.4162524 (600)	total: 14.3s	remaining: 57.3s
800:	learn: 139.9755964	test: 140.4522885	best: 140.4522885 (800)	total: 18.5s	remaining: 50.9s
1000:	learn: 134.1494004	test: 134.8201593	best: 134.8201593 (1000)	total: 23.8s	remaining: 47.6s
1200:	learn: 129.6568518	test: 130.5097899	best: 130.5097899 (1200)	total: 28.7s	remaining: 43s
1400:	learn: 125.5645233	test: 126.5819455	best: 126.5819455 (1400)	total: 33.2s	remaining: 37.9s
1600:	learn: 122.1179753	test: 123.2800521	best: 123.2800521 (1600)	total: 38.2s	remaining: 33.4s
1800:	learn: 119.1938677	test: 120.4522334	best: 120.4522334 (1800)	total: 43.5s	remaining: 29s
2000:	learn: 116.6851022	test: 118.086

In [17]:
df_test = pd.read_csv('test_WudNWDM.csv' )
df_test.head(3)

,row_id,datetime
0,94993,2019-01-01 00:00:00
1,94994,2019-01-01 01:00:00
2,94995,2019-01-01 02:00:00


In [18]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   row_id    26304 non-null  int64 
 1   datetime  26304 non-null  object
dtypes: int64(1), object(1)
memory usage: 411.1+ KB


In [19]:
df_test.rename(columns = {'datetime':'date_time'}, inplace = True)
df_test.head(3)

,row_id,date_time
0,94993,2019-01-01 00:00:00
1,94994,2019-01-01 01:00:00
2,94995,2019-01-01 02:00:00


In [20]:
df_test['year'] = df_test['date_time'].str[:4]
df_test['year'] = df_test['year'].astype('int64')

df_test['month'] = df_test['date_time'].str[5:7]
df_test['month'] = df_test['month'].astype('int64')

df_test['day'] = df_test['date_time'].str[8:10]
df_test['day'] = df_test['day'].astype('int64')

df_test['hour'] = df_test['date_time'].str[11:13]
df_test['hour'] = df_test['hour'].astype('int64')

df_test.head(3)

,row_id,date_time,year,month,day,hour
0,94993,2019-01-01 00:00:00,2019,1,1,0
1,94994,2019-01-01 01:00:00,2019,1,1,1
2,94995,2019-01-01 02:00:00,2019,1,1,2


In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   row_id     26304 non-null  int64 
 1   date_time  26304 non-null  object
 2   year       26304 non-null  int64 
 3   month      26304 non-null  int64 
 4   day        26304 non-null  int64 
 5   hour       26304 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ MB


In [22]:
df_test['date_time'] = pd.to_datetime(df_test['date_time'])

In [23]:
df_test['weekday'] = df_test['date_time'].dt.dayofweek
df_test2 = df_test.drop(['date_time', 'row_id'], axis = 1)
df_test2.head(3)

,year,month,day,hour,weekday
0,2019,1,1,0,1
1,2019,1,1,1,1
2,2019,1,1,2,1


In [24]:
df_test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   year     26304 non-null  int64
 1   month    26304 non-null  int64
 2   day      26304 non-null  int64
 3   hour     26304 non-null  int64
 4   weekday  26304 non-null  int64
dtypes: int64(5)
memory usage: 1.0 MB


In [25]:
# performing predictions on the test dataset
X_test=df_test2.to_numpy()
y_pred_test = catreg.predict(X_test)

In [26]:
y_pred_test

array([1768.78015858, 1650.81541027, 1571.03652073, ..., 2420.69005267,
       2371.21384398, 2200.99740992])

In [27]:
df_energy = pd.DataFrame(y_pred_test, columns =['energy'])
df_energy.head(3)

,energy
0,1768.780159
1,1650.815410
2,1571.036521


In [28]:
df_row_id = df_test.drop(['date_time', 'year', 'month', 'day', 'hour', 'weekday'], axis = 1)
df_row_id.head(3)

,row_id
0,94993
1,94994
2,94995


In [29]:
df_concatenated = pd.concat([df_row_id, df_energy], axis=1)
df_submission = df_concatenated
df_submission

,row_id,energy
0,94993,1768.780159
1,94994,1650.815410
2,94995,1571.036521
3,94996,1510.840389
4,94997,1501.981053
...,...,...
26299,121292,2512.951425
26300,121293,2455.352124
26301,121294,2420.690053
26302,121295,2371.213844


In [30]:
df_submission.to_csv('submission.csv')